# 모델에 필요한 파일 받기

In [1]:
#현재 위치 확인
!pwd

/content


In [2]:
#ACGPN 모델 파일 다운로드
!git clone https://github.com/kairess/ACGPN.git

Cloning into 'ACGPN'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 165 (delta 21), reused 19 (delta 18), pack-reused 141
Receiving objects: 100% (165/165), 303.15 KiB | 8.19 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [3]:
#다운로드 받은 파일의 위치로 이동
%cd ACGPN

/content/ACGPN


# dependencies 및 사전 파일 준비

In [4]:
!pip install -U --no-cache-dir gdown --pre -qq
!pip install ninja -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 5.3 MB/s eta 0:00:00


In [5]:
import gdown
import numpy as np
from PIL import Image
import IPython
import os
import sys
import time

from predict_pose import generate_pose_keypoints

In [6]:
#모델을 돌리기 위한 파일 준비
!mkdir Data_preprocessing/test_color
!mkdir Data_preprocessing/test_colormask
!mkdir Data_preprocessing/test_edge
!mkdir Data_preprocessing/test_img
!mkdir Data_preprocessing/test_label
!mkdir Data_preprocessing/test_mask
!mkdir Data_preprocessing/test_pose
!mkdir inputs
!mkdir inputs/img
!mkdir inputs/cloth

In [7]:
#현재 위치 확인
!pwd

/content/ACGPN


In [8]:
#휴먼 세그멘테이션 모델 파일 다운로드
!git clone https://github.com/levindabhi/Self-Correction-Human-Parsing-for-ACGPN.git
#옷 마스킹 모델 파일 다운로드
!git clone https://github.com/levindabhi/U-2-Net.git

Cloning into 'Self-Correction-Human-Parsing-for-ACGPN'...
remote: Enumerating objects: 769, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 769 (delta 72), reused 62 (delta 62), pack-reused 658
Receiving objects: 100% (769/769), 3.80 MiB | 14.56 MiB/s, done.
Resolving deltas: 100% (189/189), done.
Cloning into 'U-2-Net'...
remote: Enumerating objects: 822, done.
remote: Counting objects: 100% (822/822), done.
remote: Compressing objects: 100% (439/439), done.
remote: Total 822 (delta 389), reused 793 (delta 379), pack-reused 0
Receiving objects: 100% (822/822), 30.71 MiB | 34.75 MiB/s, done.
Resolving deltas: 100% (389/389), done.


# 사전학습된 모델 다운로드

# 포즈 예측 모델

In [9]:
!gdown 1hOHMFHEjhoJuLEQY0Ndurn5hfiA9mwko -O pose/pose_iter_440000.caffemodel

Downloading...
From (uriginal): https://drive.google.com/uc?id=1hOHMFHEjhoJuLEQY0Ndurn5hfiA9mwko
From (redirected): https://drive.google.com/uc?id=1hOHMFHEjhoJuLEQY0Ndurn5hfiA9mwko&confirm=t&uuid=0dfd6322-25a8-4495-ae66-e308cb9fadee
To: /content/ACGPN/pose/pose_iter_440000.caffemodel
100% 209M/209M [00:05<00:00, 40.3MB/s]


# 휴먼 세그멘테이션 모델

In [10]:
gdown.download('https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH', 'lip_final.pth', quiet=False)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
From (redirected): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH&confirm=t&uuid=bd429e60-1db3-4227-9fa5-ed8481992f2c
To: /content/ACGPN/lip_final.pth
100%|██████████| 267M/267M [00:01<00:00, 136MB/s]


'lip_final.pth'

# U-2-Net 모델 (옷 마스크 추출 모델)

In [11]:
#U-2-Net 모델을 돌릴 때 필요한 파일 준비를 위해 해당 모델 파일로 이동
%cd U-2-Net

#U-2-Net 모델에 필요한 파일 생성
!mkdir saved_models
!mkdir saved_models/u2net
!mkdir saved_models/u2netp

#사전 학습된 U-2-Net 모델 다운로드
!gdown 1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy -O saved_models/u2netp/u2netp.pth
!gdown 1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ -O saved_models/u2net/u2net.pth

import u2net_load
import u2net_run

#로드하려면 쿠다 필요 (위치 확인용)
u2net = u2net_load.model(model_name='u2netp')

#U-2-Net 모델을 위한 파일 준비가 끝나면 부모 파일로 이동
%cd ..

/content/ACGPN/U-2-Net
mkdir: cannot create directory ‘saved_models’: File exists
Downloading...
From: https://drive.google.com/uc?id=1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy
To: /content/ACGPN/U-2-Net/saved_models/u2netp/u2netp.pth
100% 4.68M/4.68M [00:00<00:00, 84.9MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ
From (redirected): https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ&confirm=t&uuid=0eca38d6-47e0-4dbc-818f-ef02629f460e
To: /content/ACGPN/U-2-Net/saved_models/u2net/u2net.pth
100% 176M/176M [00:02<00:00, 83.8MB/s]
...load U2NEP---4.7 MB
/content/ACGPN


# ACGPN모델 (사진 합성 모델)

In [12]:
#ACGPN 모델에 필요한 파일 생성
!mkdir checkpoints

#사전 학습된 ACGPN 모델 다운로드
gdown.download('https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx', output='checkpoints/ACGPN_checkpoints.zip', quiet=False)

#압축된 모델 파일 풀어서 전에 만든 파일위치에 넣어줌
!unzip checkpoints/ACGPN_checkpoints.zip -d checkpoints

Downloading...
From (uriginal): https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx
From (redirected): https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx&confirm=t&uuid=eee608f7-e754-4174-886f-dc277a59544c
To: /content/ACGPN/checkpoints/ACGPN_checkpoints.zip
100%|██████████| 524M/524M [00:04<00:00, 130MB/s]


Archive:  checkpoints/ACGPN_checkpoints.zip
   creating: checkpoints/label2city/
  inflating: checkpoints/label2city/latest_net_G.pth  
  inflating: checkpoints/label2city/latest_net_G1.pth  
  inflating: checkpoints/label2city/latest_net_G2.pth  
  inflating: checkpoints/label2city/latest_net_U.pth  
  inflating: checkpoints/label2city/opt.txt  


# 가설 1: 전처리부터 추론까지의 코드는 api안에 넣어준다.

In [13]:
!pwd

/content/ACGPN


In [14]:
!pip install colabcode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 M

In [15]:
!pip install fastapi["all"]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 474.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB

In [16]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import FileResponse, JSONResponse
import os
import time
from PIL import Image


app = FastAPI()

In [17]:

# Define the paths for the input directories and models
CLOTH_DIR = "inputs/cloth"
MODEL_DIR = "inputs/img"
PREPROCESS_DIR = "Data_preprocessing"
COLOR_DIR = f"{PREPROCESS_DIR}/test_color"
IMG_DIR = f"{PREPROCESS_DIR}/test_img"
EDGE_DIR = f"{PREPROCESS_DIR}/test_edge"
LABEL_DIR = f"{PREPROCESS_DIR}/test_label"
POSE_DIR = f"{PREPROCESS_DIR}/test_pose"
MODEL_RESTORE_PATH = "lip_final.pth"

#for test
@app.get("/")
def home():
  return 'hello this is model'

#옷 데이터 넣어주기
@app.post("/inputs/cloth")
async def upload_photo(file: UploadFile):

    UPLOAD_DIR = "./inputs/cloth"  
    
    content = await file.read()
    # 저장할 이미지 이름을 시간으로 변경
    filename = f"{int(time.time())}.png"  
    # 서버 로컬 스토리지에 이미지 저장 (쓰기)
    with open(os.path.join(UPLOAD_DIR, filename), "wb") as fp:
        fp.write(content)  
 
    return {"filename": filename}

#모델 데이터 넣어주기
@app.post("/inputs/img")
async def upload_photo(file: UploadFile):

    UPLOAD_DIR = "./inputs/img"  
    
    content = await file.read()
    # 저장할 이미지 이름을 시간으로 변경
    filename = f"{int(time.time())}.png"  
    # 서버 로컬 스토리지에 이미지 저장 (쓰기)
    with open(os.path.join(UPLOAD_DIR, filename), "wb") as fp:
        fp.write(content)  
 
    return {"filename": filename}

@app.get("/preprocess_cloth")
async def preprocess_cloth():
    sorted_cloth_files = sorted(os.listdir('inputs/cloth'))
    
    cloth_name = f'cloth_{int(time.time())}.png'

    cloth_path = os.path.join('inputs/cloth', sorted_cloth_files[0])

    cloth = Image.open(cloth_path)

    cloth = cloth.resize((192, 256), Image.BICUBIC).convert('RGB')

    cloth.save(os.path.join('Data_preprocessing/test_color', cloth_name))

    u2net_run.infer(u2net, 'Data_preprocessing/test_color', 'Data_preprocessing/test_edge')

    return cloth_name


In [23]:
#위에 적어놓음
#from fastapi.responses import JSONResponse


###################################################
#옷, 모델 데이터 넣어주기 + u2net모델 성공 but 모델은 안돌아감#
###################################################


@app.get("/preprocess_img")
async def preprocess_img():

    #모델 데이터셋이 올바른 위치에 있는지 확인
    sorted_img_files = sorted(os.listdir('inputs/img'))

    #전처리 할 모델 데이터 이름을 시간으로 지정
    img_name = f'img_{int(time.time())}.png'

    #전처리 할 모델 데이터의 파일 위치 확인
    img_path = os.path.join('inputs/img', sorted(os.listdir('inputs/img'))[0])

    #이미지 할당
    img = Image.open(img_path)

    #이미지 사이즈 조절
    img = img.resize((192,256), Image.BICUBIC)

    #전처리를 위해 모델 이미지 파일의 위치값을 해당 파일 위치에 지정해준 모델 데이터 이름으로 지정
    img_path = os.path.join('Data_preprocessing/test_img', img_name)

    #지정해준 모델 이미지 파일의 위치값을 바탕으로 이미지 저장
    img.save(img_path)

    #전처리를 위해 휴먼 세그멘테이션 모델을 parser를 이용하여 돌림 ***parser는 데이터셋의 종류, 사전학습 모델 파일의 위치, 전처리 할 모델 이미지 위치, 전처리 후 이미지 저장할 위치
    !python3 Self-Correction-Human-Parsing-for-ACGPN/simple_extractor.py --dataset 'lip' --model-restore 'lip_final.pth' --input-dir 'Data_preprocessing/test_img' --output-dir 'Data_preprocessing/test_label'

    #모델 포즈 예측 모델을 돌리고 저장할 파일 위치와 파일 이름 형식을 지정
    pose_path = os.path.join('Data_preprocessing/test_pose', img_name.replace('.png', '_keypoints.json'))

    #모델 포즈 예측 모델을 전처리 할 모델 이미지 위치, 전처리 후 파일 위치를 받아서 돌리고 저장
    generate_pose_keypoints(img_path, pose_path)
    return img_name

cloth_name = preprocess_cloth()
img_name = preprocess_img()

In [24]:
!pwd

/content/ACGPN


In [28]:
### 현재 에러 메시지
# cloth_name과 img_name 파라미터가 필요한데 없어서 422error가 발생한 것이라고 함

# {
#     "detail": [
#         {
#             "loc": [
#                 "query",
#                 "cloth_name"
#             ],
#             "msg": "field required",
#             "type": "value_error.missing"
#         },
#         {
#             "loc": [
#                 "query",
#                 "img_name"
#             ],
#             "msg": "field required",
#             "type": "value_error.missing"
#         }
#     ]
# }

@app.get("/predict")
async def predict_image():
     with open('Data_preprocessing/test_pairs.txt', 'w') as f:
          f.write(f'{img_name} {cloth_name}')

     %cd  
     os.system(f"python3 test.py")

     result_path = os.path.join('test', 'final', f'{img_name.split(".")[0]}_final.png')
     with open(result_path, "rb") as f:
        result_image = f.read()

     return result_image 

In [29]:
from colabcode import ColabCode
cc = ColabCode(port = 12000, code = False)

In [30]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://fae9-35-230-109-198.ngrok.io" -> "http://localhost:12000"


INFO:     Started server process [214]
INFO:uvicorn.error:Started server process [214]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     118.223.10.215:0 - "GET /predict HTTP/1.1" 422 Unprocessable Entity


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [214]
INFO:uvicorn.error:Finished server process [214]
